In [18]:
import numpy as np
from datasets import get_dataset
import sys 
import matplotlib.pyplot as plt
from tiffwrapper import make_composite
from decoders import get_decoder
import torch
import os
sys.path.insert(0, "../")
from model_builder import get_base_model, get_pretrained_model_v2
from configuration import Configuration 
from DEFAULTS import BASE_PATH
DATASET = "synaptic-semantic-segmentation"
MODEL = "mae-lightning-small"
WEIGHTS = "MAE_SMALL_STED"
CHECKPOINT = "/home-local/Frederic/segmentation-baselines/mae-lightning-small/factin/pretrained-MAE_SMALL_STED-46" 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
backbone, cfg = get_pretrained_model_v2(MODEL, WEIGHTS)
train_dataset, _, _ = get_dataset(name=DATASET, cfg=cfg)

in_channels 1
--- mae-lightning-small | /home-local/Frederic/baselines/mae-small_STED/pl_checkpoint-999.pth ---

--- Loaded model mae-lightning-small with weights MAE_SMALL_STED ---


100%|██████████| 1357/1357 [00:01<00:00, 1102.30it/s]

Train dataset size: 1191
Valid dataset size: 354
Test dataset size: 375


In [20]:
# class SegmentationConfiguration(Configuration):
    
#     freeze_backbone: bool = True
#     num_epochs: int = 300
#     learning_rate: float = 1e-4
# segmentation_cfg = SegmentationConfiguration()
# for key, value in segmentation_cfg.__dict__.items():
#         setattr(cfg, key, value)
# model = get_decoder(backbone, cfg)
# ckpt = torch.load(os.path.join(CHECKPOINT, "result.pt"))["model"]
# model.load_state_dict(ckpt)
# model = model.to(DEVICE)
# model.eval()

In [21]:
from tqdm import tqdm

def merge_masks(masks):
    image_rgb = make_composite(masks, luts=["green", "magenta", "cyan", "yellow"], ranges=[(masks[i].min(), masks[i].max()) for i in range(masks.shape[0])])
    return image_rgb

N = len(train_dataset)
indices = np.random.randint(0, N, size=20)

for i in tqdm(indices, total=len(indices)):
    image, mask = train_dataset[i]
    # pred = model(image.unsqueeze(0).to(DEVICE)).squeeze().detach().cpu().numpy()
    image = image.squeeze().cpu().numpy()
    mask = mask.cpu().numpy()
    mask = merge_masks(mask)
    # pred = merge_masks(pred)
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    axs[0].imshow(image, cmap="hot", vmax=1.2)
    axs[1].imshow(mask)
    # axs[2].imshow(pred)
    # axs[0].set_title(i)
    for ax in axs:
        ax.axis("off")
    fig.savefig(f"./dummy_images/{DATASET}_example_{i}.pdf", bbox_inches="tight", dpi=1200)
    plt.close(fig)
    # plt.show()


  0%|          | 0/20 [00:00<?, ?it/s]/opt/anaconda3/envs/phd-env/lib/python3.8/site-packages/tiffwrapper/utils.py:106: RuntimeWarning: invalid value encountered in divide
  scaled = (image - m[:, numpy.newaxis, numpy.newaxis]) / (M - m)[:, numpy.newaxis, numpy.newaxis]
/opt/anaconda3/envs/phd-env/lib/python3.8/site-packages/tiffwrapper/utils.py:84: RuntimeWarning: invalid value encountered in cast
  ary = ary.astype(numpy.uint8)
100%|██████████| 20/20 [02:45<00:00,  8.26s/it]
